In [2]:
!pip install gdown

In [3]:
!pip install conllu

In [4]:
import os
import gdown
import zipfile
import glob
import sqlite3
from conllu import parse
import re
import string

In [5]:
# Создание реляционной базы данных SQLite
conn = sqlite3.connect('habr_corpus.db')
cur = conn.cursor()

Наша БД будет состоять из трёх таблиц:

* articles: id статьи и метаинформация о ней (автор, заголовок, ссылка);

* sentences: id предложения, сам текст предложения, а также внешний ключ (id статьи, в которой это предложение содержится), связывающий таблицу sentences с таблицей articles;

* words: id слова, токен, лемма, часть речи, синтаксическая функция (отношение зависимости к главному слову), а также внешний ключ (id предложения, которому это слово принадлежит), связывающий таблицу words с таблицей sentences.

In [6]:
# Создание таблиц
cur.execute('''
CREATE TABLE IF NOT EXISTS articles (
    article_id INTEGER PRIMARY KEY AUTOINCREMENT,
    author TEXT,
    title TEXT,
    link TEXT
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS sentences (
    sent_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sentence TEXT,
    article_id INTEGER,
    FOREIGN KEY(article_id) REFERENCES articles(article_id)
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS words (
    word_id INTEGER PRIMARY KEY AUTOINCREMENT,
    token TEXT,
    lemma TEXT,
    pos TEXT,
    deprel TEXT,
    sent_id INTEGER,
    FOREIGN KEY(sent_id) REFERENCES sentences(sent_id)
)
''')
conn.commit()

In [7]:
url = 'https://drive.google.com/uc?id=1O38w8WUD-JPNHerPWNc5l69iZNtX4HTx'
output = 'conllu.zip'

# Скачивание zip-файла с .conllu-файлами
gdown.download(url, output, quiet=False)

# Распаковка zip-файла
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('conllu')

# Удаление zip-файла после распаковки
os.remove(output)

Downloading...
From (original): https://drive.google.com/uc?id=1O38w8WUD-JPNHerPWNc5l69iZNtX4HTx
From (redirected): https://drive.google.com/uc?id=1O38w8WUD-JPNHerPWNc5l69iZNtX4HTx&confirm=t&uuid=bc1b661d-b92b-4638-a6c3-e7a8b2dc9ebf
To: /content/conllu.zip
100%|██████████| 41.3M/41.3M [00:00<00:00, 74.0MB/s]


In [8]:
directory = "/content/conllu/conllu"

article_id = 1
sent_id = 1
word_id = 1

# Обработка каждого .conllu-файла и сохранение данных в таблицы
for file in os.listdir(directory):
  file_path = os.path.join(directory, file)
  with open(file_path, 'r', encoding='utf-8') as f:
    data = f.read()
  sentences = parse(data)
  metainfo = sentences[0].metadata['metadata']
  pattern = r'автор: (.*), заголовок: (.*), ссылка: (.*)'
  author = re.search(pattern, metainfo).group(1)
  title = re.search(pattern, metainfo).group(2)
  link = re.search(pattern, metainfo).group(3)
  cur.execute("INSERT INTO articles VALUES (?,?,?,?)", (article_id, author, title, link))
  for sent in sentences:
    sentence = sent.metadata['text']
    cur.execute("INSERT INTO sentences VALUES (?,?,?)", (sent_id, sentence, article_id))
    for word in sent:
      token = word['form'].lower().strip(string.punctuation)
      pos = word['upos']
      if pos != 'PUNCT' and pos != 'SYM' and pos != 'X' and not any(char.isdigit() for char in token):
        lemma = word['lemma'].lower().strip(string.punctuation)
        deprel = word['deprel']
        cur.execute("INSERT INTO words VALUES (?,?,?,?,?,?)", (word_id, token, lemma, pos, deprel, sent_id))
        word_id += 1
    sent_id += 1
  article_id += 1
conn.commit()

In [9]:
# Создание индексов для повышения скорости и эффективности запросов к базе данных
cur.executescript('''
CREATE INDEX idx_token ON words(token);
CREATE INDEX idx_lemma ON words(lemma);
CREATE INDEX idx_pos ON words(pos);
CREATE INDEX idx_deprel ON words(deprel);
''')
conn.commit()

In [10]:
# Закрытие соединения
conn.close()